In [72]:
from abc import ABC, abstractmethod
from typing import List
from enum import Enum

class Visitor:
    pass

class Component(ABC):
    """
    The Component interface declares an `accept` method that should take the
    base visitor interface as an argument.
    """

    @abstractmethod
    def accept(self, visitor: Visitor) -> None:
        pass

class ConcreteComponentA(Component):
    """
    Each Concrete Component must implement the `accept` method in such a way
    that it calls the visitor's method corresponding to the component's class.
    """

    def accept(self, visitor: Visitor) -> None:
        """
        Note that we're calling `visitConcreteComponentA`, which matches the
        current class name. This way we let the visitor know the class of the
        component it works with.
        """

        visitor.visit_concrete_component_a(self)

    def exclusive_method_of_concrete_component_a(self) -> str:
        """
        Concrete Components may have special methods that don't exist in their
        base class or interface. The Visitor is still able to use these methods
        since it's aware of the component's concrete class.
        """

        return "A"


class ConcreteComponentB(Component):
    """
    Same here: visitConcreteComponentB => ConcreteComponentB
    """

    def accept(self, visitor: Visitor):
        visitor.visit_concrete_component_b(self)

    def special_method_of_concrete_component_b(self) -> str:
        return "B"

class Visitor(ABC):
    """
    The Visitor Interface declares a set of visiting methods that correspond to
    component classes. The signature of a visiting method allows the visitor to
    identify the exact class of the component that it's dealing with.
    """

    @abstractmethod
    def visit_concrete_component_a(self, element: ConcreteComponentA) -> None:
        pass

    @abstractmethod
    def visit_concrete_component_b(self, element: ConcreteComponentB) -> None:
        pass


"""
Concrete Visitors implement several versions of the same algorithm, which can
work with all concrete component classes.

You can experience the biggest benefit of the Visitor pattern when using it with
a complex object structure, such as a Composite tree. In this case, it might be
helpful to store some intermediate state of the algorithm while executing
visitor's methods over various objects of the structure.
"""


class ConcreteVisitor1(Visitor):
    def visit_concrete_component_a(self, element) -> None:
        print(f"{element.exclusive_method_of_concrete_component_a()} + ConcreteVisitor1")

    def visit_concrete_component_b(self, element) -> None:
        print(f"{element.special_method_of_concrete_component_b()} + ConcreteVisitor1")


class ConcreteVisitor2(Visitor):
    def visit_concrete_component_a(self, element) -> None:
        print(f"{element.exclusive_method_of_concrete_component_a()} + ConcreteVisitor2")

    def visit_concrete_component_b(self, element) -> None:
        print(f"{element.special_method_of_concrete_component_b()} + ConcreteVisitor2")


def client_code(components: List[Component], visitor: Visitor) -> None:
    """
    The client code can run visitor operations over any set of elements without
    figuring out their concrete classes. The accept operation directs a call to
    the appropriate operation in the visitor object.
    """

    # ...
    for component in components:
        component.accept(visitor)
    # ...

components = [ConcreteComponentA(), ConcreteComponentB()]

print("The client code works with all visitors via the base Visitor interface:")
visitor1 = ConcreteVisitor1()
client_code(components, visitor1)

print("It allows the same client code to work with different types of visitors:")
visitor2 = ConcreteVisitor2()
client_code(components, visitor2)

The client code works with all visitors via the base Visitor interface:
A + ConcreteVisitor1
B + ConcreteVisitor1
It allows the same client code to work with different types of visitors:
A + ConcreteVisitor2
B + ConcreteVisitor2


In [73]:
from pyleri import (
    Choice,
    Grammar,
    Keyword,
    List,
    Optional,
    Regex,
    Repeat,
    Ref,
    Sequence)

import json

class CircuitJSGrammar(Grammar):
    START = Ref()
    
    number_literal = Regex('[-+]?[0-9]*\.?[0-9]+([eE][-+]?[0-9]+)?')
    two_terminal_coords = Repeat(number_literal, mi=4, ma=4)
    booly = Choice(Keyword('true'), Keyword('false'))

    capacitor = Sequence(Keyword('c'), two_terminal_coords, number_literal, number_literal, Repeat(number_literal, mi=2, ma=2))
    ground = Sequence(Keyword('g'), two_terminal_coords, Repeat(number_literal, mi=2, ma=2))    
    inductor = Sequence(Keyword('l'),  two_terminal_coords, number_literal, number_literal, Repeat(number_literal, mi=2, ma=2))
    resistor = Sequence(Keyword('r'), two_terminal_coords, number_literal, number_literal)
    switch = Sequence(Keyword('s'), two_terminal_coords, number_literal, number_literal, booly)
    voltage = Sequence(Keyword('v'), two_terminal_coords, Repeat(number_literal, mi=3, ma=3), number_literal, Repeat(number_literal, mi=2, ma=2), number_literal)
    wire = Sequence(Keyword('w'), two_terminal_coords, number_literal)

    START = Choice(capacitor, ground, inductor, resistor, switch, voltage, wire)
   
def node_props(node, children):
    return {
        'start': node.start,
        'end': node.end,
        'name': node.element.name if hasattr(node.element, 'name') else None,
        'element': node.element.__class__.__name__,
        'string': node.string,
        'children': children
    }


# Recursive method to get the children of a node object:
def get_children(children):
    return [node_props(c, get_children(c.children)) for c in children]


# View the parse tree:
def view_parse_tree(res):
    start = res.tree.children[0] \
        if res.tree.children else res.tree
    return node_props(start, get_children(start.children))
        
grammar = CircuitJSGrammar()
#print(grammar.parse('r 240 80 448 80 0 10').is_valid)
#res = grammar.parse('r 240 80 448 80 0 10')
#print(json.dumps(view_parse_tree(res), indent=2))
#print(grammar.parse('v 32 288 32 224 0 0 40 12 0 0 0.5').is_valid)

#res = grammar.parse('c 368 64 368 144 0 1e-7 0.001 0.001')
#print(res.is_valid)
#print(res.pos)


In [74]:
class ComponentWarehouse:
    def __init__(self):
        self.classes = {}

    def add_class(self, c):
        self.classes[c.__name__] = c

    # -- the decorator
    def component(self, c):
        self.add_class(c)

        # Decorators have to return the function/class passed (or a modified variant thereof), however I'd rather do this separately than retroactively change add_class, so.
        # "held" is more succint, anyway.
        return c 

    def __getitem__(self, n):
        return self.classes[n]

component_warehouse = ComponentWarehouse()

In [75]:
class ElectricComponent(object):
    
    id = 1

    def __init__(self):
        self._direction = 'right'

    @property
    def direction(self):
        return self._direction

    @direction.setter
    def direction(self, direction):
        self._direction = direction

    @property
    def hasValue(self):
        return True

    @property
    def hasLabel(self):
        return True

    def accept(v: Visitor) -> None:
        v.visit_any(self)

    

In [76]:
class Visitor(ABC):

    def __init__(self, drawingContext):
        self.d = drawingContext

    @abstractmethod
    def visit_any(self, element: ElectricComponent) -> None:
        pass

    @abstractmethod
    def visit_three_terminal(self, element: ElectricComponent) -> None:
        pass

In [77]:

class TwoTerminalComponent(ElectricComponent):
    def setValue(self, parsing_element):
        self._value = parsing_element[3].string
        print(self._value)
        
class TwoTerminalDirectionalComponent(TwoTerminalComponent):
    pass

class ThreeTerminalComponent(ElectricComponent):
    pass


In [78]:
@component_warehouse.component
class capacitor(TwoTerminalComponent):
    class Units(Enum):
        farads = "F"
        microfarads = "µF"
        picofarads = "pF"

    def getElement(self):
        return elm.Capacitor

    @property
    def labelPrefix(self):
        return "C"

    def accept(v: Visitor) -> None:
        v.visit_any(self)

@component_warehouse.component
class ground(ElectricComponent):
    def getElement(self):
        return elm.Ground

    @property
    def hasValue(self):
        return False

    @property
    def hasLabel(self):
        return False

@component_warehouse.component
class resistor(TwoTerminalComponent):
    class Units(Enum):
        ohms = "Ω"
        killohms = "kΩ"
        megaohms = "mΩ"

    def getElement(self):
        return elm.Resistor

    @property
    def labelPrefix(self):
        return "R"


@component_warehouse.component
class inductor(TwoTerminalComponent):
    class Units(Enum):
        henry = "H"
        millihenry = "mH"
        microhenry = "µH"
    
    def getElement(self):
        return elm.Inductor

@component_warehouse.component
class switch(TwoTerminalComponent):
    def getElement(self):
        return elm.Switch

    @property
    def hasValue(self):
        return False

    @property
    def hasLabel(self):
        return False    

@component_warehouse.component
class voltage(TwoTerminalComponentDirectional):
    class Units(Enum):
        volts = "V"

    def getElement(self):
        return elm.SourceV

    @property
    def labelPrefix(self):
        return "V"

    @property
    def isDirectional(self):
        return True

@component_warehouse.component
class wire(ElectricComponent):
    def getElement(self):
        return elm.Line

    @property
    def hasValue(self):
        return False

    @property
    def hasLabel(self):
        return False




In [79]:
class SchemDrawVisitor(Visitor):    
    def visit_any(self, element: ElectricComponent) -> None:
        pass

    def visit_three_terminal(self, element: ElectricComponent) -> None:
        pass